In [1]:
import re
import nltk
import heapq
import requests
import bs4 as bs
from nltk.corpus import stopwords

In [2]:
link = 'https://es.wikipedia.org/wiki/National_Basketball_Association'

In [3]:
source = requests.get(link)
print(source.status_code)

200


In [4]:
# Si todo va bien, vamos a extraer el texto del documento
soup = bs.BeautifulSoup(source.content, 'lxml')
print(soup)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="es">
<head>
<meta charset="utf-8"/>
<title>National Basketball Association - Wikipedia, la enciclopedia libre</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"d11ee85e-81f6-4a44-b24d-28dc20ea3589","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"National_Basketball_Association","wgTitle":"National Basketball Association","wgCurRevisionId":145935074,"wgRevisionId":145935074,"wgArticleId":75919,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Artículos con datos por trasladar a Wikidata","Wikipedia:Ar

In [5]:
# En Wikipedia el texto se guarda por parrafos en las etiquetas <p>
texto = ''

for parrafo in soup.find_all('p'):
    texto+= parrafo.text

# Limpieando el texto
texto = re.sub(r'\n|\u200b|\t', '', texto)
texto = re.sub(r'\[[0-9]]*\]+', '', texto)
texto = re.sub(r'\+|\(|\)|\-|\#', '', texto)
# Dejamos todo en minusculas
texto = texto.lower()

texto

'la national basketball association, más conocida simplemente por sus siglas nba, es una liga privada de baloncesto profesional que se disputa en estados unidos desde 1949, cuando se fusionaron las ligas profesionales national basketball league nbl, creada en 1937 y la basketball association of america baa, fundada en 1946. al ser una entidad privada, no depende oficialmente de la federación estadounidense de baloncesto. los jugadores de la nba están autorizados a competir internacionalmente tras un acuerdo especial a tres bandas firmado entre la federación internacional de baloncesto fiba —máximo organismo a nivel mundial—, la propia nba y la federación estadounidense en 1989.fue al término de la segunda guerra mundial cuando un grupo de hombres —en su mayoría propietarios de algunos de los recintos deportivos de diferentes ciudades estadounidenses—, se reunieron en el madison square garden de nueva york con el fin de buscar alternativas para llenar dichos pabellones con algo más que 

In [6]:
# Hacemos una limpieza más profunda de nuestro texto
texto_limpio = re.sub(r'\W', ' ', texto) # Quitar signos de puntuación
texto_limpio = re.sub(r'\d', ' ', texto_limpio) # Quitar dígitos
texto_limpio = re.sub(r'\s+', ' ', texto_limpio) # Cambiar los que son 2 o más espacios por 1
texto_limpio

'la national basketball association más conocida simplemente por sus siglas nba es una liga privada de baloncesto profesional que se disputa en estados unidos desde cuando se fusionaron las ligas profesionales national basketball league nbl creada en y la basketball association of america baa fundada en al ser una entidad privada no depende oficialmente de la federación estadounidense de baloncesto los jugadores de la nba están autorizados a competir internacionalmente tras un acuerdo especial a tres bandas firmado entre la federación internacional de baloncesto fiba máximo organismo a nivel mundial la propia nba y la federación estadounidense en fue al término de la segunda guerra mundial cuando un grupo de hombres en su mayoría propietarios de algunos de los recintos deportivos de diferentes ciudades estadounidenses se reunieron en el madison square garden de nueva york con el fin de buscar alternativas para llenar dichos pabellones con algo más que hockey sobre hielo y boxeo los cua

In [7]:
# Implementamos el modelo de sumarización
# Al ya no tener puntos, sacaremos las frases del texto original
frases = nltk.sent_tokenize(texto, language='spanish')
frases

['la national basketball association, más conocida simplemente por sus siglas nba, es una liga privada de baloncesto profesional que se disputa en estados unidos desde 1949, cuando se fusionaron las ligas profesionales national basketball league nbl, creada en 1937 y la basketball association of america baa, fundada en 1946. al ser una entidad privada, no depende oficialmente de la federación estadounidense de baloncesto.',
 'los jugadores de la nba están autorizados a competir internacionalmente tras un acuerdo especial a tres bandas firmado entre la federación internacional de baloncesto fiba —máximo organismo a nivel mundial—, la propia nba y la federación estadounidense en 1989.fue al término de la segunda guerra mundial cuando un grupo de hombres —en su mayoría propietarios de algunos de los recintos deportivos de diferentes ciudades estadounidenses—, se reunieron en el madison square garden de nueva york con el fin de buscar alternativas para llenar dichos pabellones con algo más

In [8]:
# Recuperamos del diccionario de StopWords
stopwords = stopwords.words('spanish')

In [9]:
# Creamos nuestra tabla para las frecuencias de las palabras
# Diccionario {palabra: frecuencia}
word2count = {}

# Haciendo una lista con nuestras palabras
words = nltk.word_tokenize(texto_limpio, language='spanish')

# Ciclo para hacer nuestro diccionario de frecuencia
for word in words:
    # Quitando las StopWords
    if word not in stopwords:
        # Checando si la palabra está en el diccionario
        # En caso de que no esté
        if word not in word2count.keys():
            word2count[word] = 1
        # En caso de que sí esté
        else:
            word2count[word] += 1

word2count

{'national': 6,
 'basketball': 10,
 'association': 6,
 'conocida': 1,
 'simplemente': 1,
 'siglas': 1,
 'nba': 71,
 'liga': 30,
 'privada': 2,
 'baloncesto': 8,
 'profesional': 2,
 'disputa': 4,
 'unidos': 7,
 'fusionaron': 1,
 'ligas': 9,
 'profesionales': 3,
 'league': 7,
 'nbl': 4,
 'creada': 1,
 'of': 3,
 'america': 4,
 'baa': 8,
 'fundada': 2,
 'ser': 6,
 'entidad': 1,
 'depende': 1,
 'oficialmente': 2,
 'federación': 3,
 'estadounidense': 7,
 'jugadores': 36,
 'autorizados': 1,
 'competir': 1,
 'internacionalmente': 1,
 'tras': 9,
 'acuerdo': 3,
 'especial': 1,
 'tres': 15,
 'bandas': 1,
 'firmado': 1,
 'internacional': 1,
 'fiba': 1,
 'máximo': 6,
 'organismo': 1,
 'nivel': 7,
 'mundial': 4,
 'propia': 2,
 'término': 3,
 'segunda': 6,
 'guerra': 2,
 'grupo': 1,
 'hombres': 1,
 'mayoría': 3,
 'propietarios': 3,
 'recintos': 1,
 'deportivos': 2,
 'diferentes': 4,
 'ciudades': 7,
 'estadounidenses': 4,
 'reunieron': 1,
 'madison': 2,
 'square': 2,
 'garden': 2,
 'nueva': 7,
 'york'

In [10]:
# Obtener la frecuencia más alta para normalizar las frecuencias
max_frec = max(word2count.values())

# Normalizando
for word in word2count.keys():
    word2count[word] = word2count[word] / max_frec
word2count

{'national': 0.08450704225352113,
 'basketball': 0.14084507042253522,
 'association': 0.08450704225352113,
 'conocida': 0.014084507042253521,
 'simplemente': 0.014084507042253521,
 'siglas': 0.014084507042253521,
 'nba': 1.0,
 'liga': 0.4225352112676056,
 'privada': 0.028169014084507043,
 'baloncesto': 0.11267605633802817,
 'profesional': 0.028169014084507043,
 'disputa': 0.056338028169014086,
 'unidos': 0.09859154929577464,
 'fusionaron': 0.014084507042253521,
 'ligas': 0.1267605633802817,
 'profesionales': 0.04225352112676056,
 'league': 0.09859154929577464,
 'nbl': 0.056338028169014086,
 'creada': 0.014084507042253521,
 'of': 0.04225352112676056,
 'america': 0.056338028169014086,
 'baa': 0.11267605633802817,
 'fundada': 0.028169014084507043,
 'ser': 0.08450704225352113,
 'entidad': 0.014084507042253521,
 'depende': 0.014084507042253521,
 'oficialmente': 0.028169014084507043,
 'federación': 0.04225352112676056,
 'estadounidense': 0.09859154929577464,
 'jugadores': 0.5070422535211268,

In [11]:
# Asignamos un score a las frases del texto original
# Diccionario de la forma {Frase: valor}
sent2count = {}

# Ciclo para ir de frase en frase
for frase in frases:
    # Hacemos una lista de las palabras
    for word in nltk.word_tokenize(frase.lower()):
        # Checamos que la palabra esté en el diccionario
        if word in word2count.keys():
            # Si la frase es más pequeña de 20 palabras
            if len(frase.split(' ')) < 20:
                # Si no está creada la frase en el diccionario la agrega
                if frase not in sent2count.keys():
                    sent2count[frase] = word2count[word]
                # Si ya está, no más suma 1
                else:
                    sent2count[frase] += word2count[word]

In [12]:
# Con el diccionario de frases vamos a seleccionar las que tengan el mayor score para agregarlas al resumen
frases_importantes = heapq.nlargest(20, sent2count, key=sent2count.get)

# Imprimimos las frases importantes
print('========== RESUMEN ==========')
print()
for frase in frases_importantes:
    print(frase)

========== RESUMEN ==========

[39] nba tv transmite partidos seleccionados cada temporada.
el equipo de florida consiguió vencer en cinco partidos, conquistando su segundo título de la nba en seis años.
en ella cada uno de los equipos de la liga tiene derecho a elegir un jugador en principio.
[33]en la ronda final, se enfrentan los campeones de cada conferencia al mejor de siete partidos.
[17]en 1984, michael jordan empezó a jugar en la nba, provocando un mayor interés en la liga.
el primero que consiga vencer en cuatro partidos, se le conocerá como el campeón de la nba.
a cada jugador se le entrega un anillo de campeón que representa el título logrado por el equipo.
la franquicia con mejor balance tiene la ventaja de campo en cada serie de playoffs, incluida la primera ronda.
además, sporting news concede anualmente un premio no oficial pero extensamente aprobado al mejor ejecutivo de la temporada.
la posición de los cuatro equipos restantes viene determinada por el porcentaje de vic